In [1]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
from unicodedata import normalize
import pandas as pd
import json

In [2]:
job = input("Введите профессию: ").encode('utf-8')

Введите профессию: водитель


In [3]:
url = 'https://barnaul.hh.ru/search/vacancy'

In [4]:
page = 0

In [5]:
params = {'text': job,
          'clusters': 'true',
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': '',
          'page': page}

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [6]:
response = requests.get(url, params=params, headers=headers)

In [7]:
dom = bs(response.text, 'html.parser')

In [8]:
vacantions = dom.find_all('a', {'class': 'bloko-link', 'data-qa': 'vacancy-serp__vacancy-title'})

In [9]:
salaries = dom.find_all('span', {'data-qa': "vacancy-serp__vacancy-compensation"})

In [10]:
vacantion_list = []

In [11]:
if response.ok and vacantions:
    for vacantion in vacantions:
        vacantion_data = {}
        link = vacantion['href']
        name = vacantion.string
        vacantion_data['name'] = name
        vacantion_data['link'] = link
        vacantion_list.append(vacantion_data)

In [12]:
len(vacantions), len(salaries), len(vacantion_list)

(20, 19, 20)

In [13]:
for salary in salaries:
    salary_data = {}
    wage = salary.text
    stroka = wage.replace("\u202f", "")
    nums = re.findall(r'\d+', stroka)
    nums = [int(i) for i in nums]
    valuta = re.findall(r'\D+$', stroka)[0]
    valuta = normalize('NFKD', valuta)
    #salary_data['valuta'] = valuta
    
    if len(nums) == 2:
        salary_data['salary_min'] = nums[0]
        salary_data['salary_max'] = nums[1]
        salary_data['valuta'] = valuta
    elif len(nums) == 1 and stroka.find('от') == -1:
        salary_data['salary_min'] = None
        salary_data['salary_max'] = nums[0]
        salary_data['valuta'] = valuta
    elif len(nums) == 1 and stroka.find('до') == -1:
        salary_data['salary_min'] = nums[0]
        salary_data['salary_max'] = None
        salary_data['valuta'] = valuta
    else:
        salary_data['salary_min'] = None
        salary_data['salary_max'] = None
        salary_data['valuta'] = valuta
    
    vacantion_list.append(salary_data)

In [14]:
#salaries

In [15]:
#vacantion_list

In [16]:
for i in range(int(len(vacantion_list))):
    if i < int((len(vacantion_list))/2):
        vacantion_list[i].update(vacantion_list[int(i+(int(len(vacantion_list)))/2)])

In [17]:
vacantion_list = vacantion_list[:int(len(vacantion_list)/2)]

In [18]:
len(vacantion_list)

19

In [19]:
vacantion_list

[{'name': 'Водитель-курьер на личном автомобиле',
  'link': 'https://barnaul.hh.ru/vacancy/43406077?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C'},
 {'name': 'Водитель-экспедитор категории С',
  'link': 'https://hhcdn.ru/click?b=239287&place=36',
  'salary_min': 73000,
  'salary_max': 86000,
  'valuta': ' руб.'},
 {'name': 'Водитель на служебном автомобиле',
  'link': 'https://barnaul.hh.ru/vacancy/49143592?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
  'salary_min': 77000,
  'salary_max': None,
  'valuta': ' руб.'},
 {'name': 'Водитель-курьер на личном автомобиле',
  'link': 'https://barnaul.hh.ru/vacancy/44484763?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C',
  'salary_min': 70000,
  'salary_max': 70000,
  'valuta': ' руб.'},
 {'name': 'Водитель-экспедитор на автомобиле компании',
  'link': 'https://barnaul.hh.ru/vacancy/47659314?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4

In [20]:
with open('2.1.json', 'w') as write_f:
    json.dump(vacantion_list, write_f)

In [21]:
df = pd.DataFrame(vacantion_list)

In [22]:
pd.set_option('display.max_columns', None)

In [23]:
df

,name,link,salary_min,salary_max,valuta
0,Водитель-курьер на личном автомобиле,https://barnaul.hh.ru/vacancy/43406077?from=va...,NaN,NaN,NaN
1,Водитель-экспедитор категории С,https://hhcdn.ru/click?b=239287&place=36,73000.0,86000.0,руб.
2,Водитель на служебном автомобиле,https://barnaul.hh.ru/vacancy/49143592?from=va...,77000.0,NaN,руб.
3,Водитель-курьер на личном автомобиле,https://barnaul.hh.ru/vacancy/44484763?from=va...,70000.0,70000.0,руб.
4,Водитель-экспедитор на автомобиле компании,https://barnaul.hh.ru/vacancy/47659314?from=va...,120000.0,NaN,руб.
5,Водитель-курьер на автомобиле компании,https://barnaul.hh.ru/vacancy/47994021?from=va...,65000.0,100000.0,руб.
6,Водитель-курьер (торговый представитель) на ав...,https://barnaul.hh.ru/vacancy/41094027?from=va...,120000.0,140000.0,руб.
7,Водитель,https://barnaul.hh.ru/vacancy/48470198?from=va...,80500.0,148000.0,руб.
8,Водитель-курьер на служебном автомобиле Ford T...,https://barnaul.hh.ru/vacancy/49120401?from=va...,120000.0,NaN,руб.
9,Водитель-экспедитор,https://barnaul.hh.ru/vacancy/48344891?from=va...,100000.0,130000.0,руб.


In [24]:
from IPython.display import HTML
HTML(df.to_html(escape=False))

,name,link,salary_min,salary_max,valuta
0,Водитель-курьер на личном автомобиле,https://barnaul.hh.ru/vacancy/43406077?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,NaN,NaN,NaN
1,Водитель-экспедитор категории С,https://hhcdn.ru/click?b=239287&place=36,73000.0,86000.0,руб.
2,Водитель на служебном автомобиле,https://barnaul.hh.ru/vacancy/49143592?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,77000.0,NaN,руб.
3,Водитель-курьер на личном автомобиле,https://barnaul.hh.ru/vacancy/44484763?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,70000.0,70000.0,руб.
4,Водитель-экспедитор на автомобиле компании,https://barnaul.hh.ru/vacancy/47659314?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.0,NaN,руб.
5,Водитель-курьер на автомобиле компании,https://barnaul.hh.ru/vacancy/47994021?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,65000.0,100000.0,руб.
6,Водитель-курьер (торговый представитель) на автомобиле компании,https://barnaul.hh.ru/vacancy/41094027?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.0,140000.0,руб.
7,Водитель,https://barnaul.hh.ru/vacancy/48470198?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,80500.0,148000.0,руб.
8,Водитель-курьер на служебном автомобиле Ford Transit,https://barnaul.hh.ru/vacancy/49120401?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.0,NaN,руб.
9,Водитель-экспедитор,https://barnaul.hh.ru/vacancy/48344891?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,100000.0,130000.0,руб.


In [25]:
df["hypertext"] = df["link"].map('<a href="{}">'.format) + df["name"] + "</a>"

In [26]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val,val)

df = df.style.format({'url': make_clickable})

In [27]:
df = df.hide_columns('name')
df

,link,salary_min,salary_max,valuta,hypertext
0,https://barnaul.hh.ru/vacancy/43406077?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,nan,nan,nan,Водитель-курьер на личном автомобиле
1,https://hhcdn.ru/click?b=239287&place=36,73000.000000,86000.000000,руб.,Водитель-экспедитор категории С
2,https://barnaul.hh.ru/vacancy/49143592?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,77000.000000,nan,руб.,Водитель на служебном автомобиле
3,https://barnaul.hh.ru/vacancy/44484763?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,70000.000000,70000.000000,руб.,Водитель-курьер на личном автомобиле
4,https://barnaul.hh.ru/vacancy/47659314?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.000000,nan,руб.,Водитель-экспедитор на автомобиле компании
5,https://barnaul.hh.ru/vacancy/47994021?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,65000.000000,100000.000000,руб.,Водитель-курьер на автомобиле компании
6,https://barnaul.hh.ru/vacancy/41094027?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.000000,140000.000000,руб.,Водитель-курьер (торговый представитель) на автомобиле компании
7,https://barnaul.hh.ru/vacancy/48470198?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,80500.000000,148000.000000,руб.,Водитель
8,https://barnaul.hh.ru/vacancy/49120401?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,120000.000000,nan,руб.,Водитель-курьер на служебном автомобиле Ford Transit
9,https://barnaul.hh.ru/vacancy/48344891?from=vacancy_search_list&query=%D0%B2%D0%BE%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C,100000.000000,130000.000000,руб.,Водитель-экспедитор
